# DataUt 
## Extracting data from Datainn and puting it to use in Pandas dataframes

In [1]:
# Import of needed libraries

# Libraries for retieving data from the GraphQL API
import urllib3
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Imports for plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams["figure.figsize"] = [14, 7]

import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(14, 7)})
sns.set_context("paper")

# Magic stuff from the Pandas people, flatteing the JSON files into something that can pass off as a table
from pandas.io.json import json_normalize

# Tools to work with dates
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz


# Remove warnings related to HTTPS, no need as data is public
urllib3.disable_warnings()

# And finaly the Pandas library for working with the data 
import pandas as pd

# Fixing filesnames that include wierd chars
from slugify import slugify


In [2]:
# Setup of the GQL client to talk to the NPRA Datainn API

sample_transport=RequestsHTTPTransport(
    url='https://www.vegvesen.no/trafikkdata/api/',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    retries=3,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

In [3]:
# Function to extract daily volumes from one day to the other

def get_day(startday, stopday, point_id):
    # Dateformat: "2010-01-01T00:00:00+02:00"
    #
    # This functions works slowly - the pagination stuff in GQL is rather code intensive. Som this just brute force
    # the whole idea of pagination by sending a call for every day. If any GQL experts want to fix this please do 
    # and let me know

    query_text_template = '''{trafficData(trafficRegistrationPointId: "{point_id}") {
        volume {
          byDay(from: "{text_page_start}", to: "{text_page_end}") {
            edges {
              node {
                from
                to
                total {
                  volumeNumbers {
                    volume
                  }
                  coverage {
                    percentage
                  }
                }
              }
            }
          }
        }
      }
    }'''


    # Add point ID into template
    query_text_template = query_text_template.replace("{point_id}", point_id)
    
    start= datetime.fromisoformat(startday)
    stop = datetime.fromisoformat(stopday)

    page_start = start
    final_stop = stop

    #Create an empty dataframe with only column names
    df = pd.DataFrame(columns=['from', 'to', 'daily_volume', 'coverage'])

    while page_start < final_stop:
        page_stop = page_start + relativedelta(days=1)

        #Entering start end dates into query text
        query = query_text_template.replace("{text_page_start}", page_start.isoformat(sep='T'))
        query = query.replace("{text_page_end}", page_stop.isoformat(sep='T'))

        #Run query to get data and create temporary dataframe
        Q = gql(query)
        data = client.execute(Q)
        temp_df = json_normalize(data, record_path=['trafficData', 'volume', 'byDay', 'edges'])

        # Fix problem when there is no data
        if len(temp_df.index) > 0:
            column_names = ['from', 'to', 'daily_volume', 'coverage']
            temp_df.columns=column_names
            df = pd.concat([df, temp_df])

        # make move to next page
        page_start = page_stop
    df.set_index('from', inplace = True, drop=False )
    df['csum'] = df['daily_volume'].cumsum()
    return df



In [4]:

def plott_anual_comparison_to_pdf(startdate, enddate, point, point_name):
    '''
    startdata and enddate is datetime object
    example point id: "35002V72811"
    '''
    timezone = pytz.timezone("Europe/Oslo")
    
    startdate = timezone.localize(startdate)
    enddate = timezone.localize(enddate)
            
    start2019 = startdate - pd.DateOffset(years=1)
    end2019 = enddate - pd.DateOffset(years=1)
        
    volume_2020 = get_day(startdate.isoformat(sep='T'), enddate.isoformat(sep='T'), point)
    volume_2019 = get_day(start2019.isoformat(sep='T'), end2019.isoformat(sep='T'), point)
    
    #Shifting the 2019 data one year forward and plotting
    volume_2020['fra'] = pd.to_datetime(volume_2020['from'])
    volume_2020.set_index('fra', inplace = True, drop=False )

    volume_2019['fra'] = pd.to_datetime(volume_2019['from']) + pd.DateOffset(years=1)
    volume_2019.set_index('fra', inplace = True, drop=False )

    # Plot setup
    plt.figure(figsize=(15,7))
    
    # Plotting of the recieved data
    plt.plot(volume_2020['csum'], label='2020 Data')
    plt.plot(volume_2019['csum'], label='2019 Data')

    # Adding a legend
    title = point + '  -  ' + point_name
       
    plt.suptitle(title)
    plt.title( "Statens vegvesen - Transportutvikling, Transportstyring")
    
    plt.xlabel('Dato')
    plt.ylabel("Total antall kjøretøy")
    plt.legend()
    
    point_name = slugify(point_name)
    filename = point_name + '.pdf'
    plt.savefig(filename)
    plt.close()


In [5]:
start =  datetime(2020, 1, 1)
end =   datetime(2020, 4, 1)
plott_anual_comparison_to_pdf(start, end, "35002V72811", "Trondheim")

/home/tlev/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [6]:
# Restore list of vehicle counting point from Vehicle_Counting_point notebook

%store -r vehicle_cointing_points
vehicle_cointing_points.head()

,id,name,lat,lon,road_reference,road_category
0,56100V804816,RAMPE SØRÅS MOT HOP,60.325803,5.337152,EV39 S78D1 m6434 KD3 m172,Europaveg
1,03486V319647,ØLEN/ETNE,59.623114,5.886108,EV134 S4D1 m17825,Europaveg
2,20318V625294,FESTNINGTUNNEL,59.911180,10.724291,EV18 S55D1 m5076,Europaveg
3,00030V705190,MATRAND S,60.020618,12.113421,RV2 S2D1 m4510,Riksveg
4,29614V805708,SOTRABRUA VEST,60.372022,5.155573,RV555 S1D1 m11739,Riksveg


In [ ]:

start =  datetime(2020, 1, 1)
end =   datetime(2020, 4, 1)

for i in range(len(vehicle_cointing_points)) : 
    
    id =vehicle_cointing_points.loc[i, "id"]
    name = vehicle_cointing_points.loc[i, "name"]
    road_reference = vehicle_cointing_points.loc[i, "road_reference"]
    
    title = road_reference + '\n' + name
    
    print(id + '  -  ' + name ) 
    plott_anual_comparison_to_pdf(start, end, id, title)

56100V804816  -  RAMPE SØRÅS MOT HOP
03486V319647  -  ØLEN/ETNE
20318V625294  -  FESTNINGTUNNEL
00030V705190  -  MATRAND S
29614V805708  -  SOTRABRUA VEST
02087V625292  -  EV 18 V/ MASTEMYR
62464V2725991  -  Pårampe fra Eiganestunellen mot Hundvåg
76092V705200  -  ALVDAL
31504V578608  -  SNÅSAHEIA
42004V22151  -  Sprøkilen
70741V384496  -  NAUSTDALSTUNNELEN
05960V384024  -  REED VEST
63227V885182  -  Skitdalshøgda
37055V971362  -  MOENKRYSSET ØST
10239V2725979  -  Hundvågtunellen fra Gamlingen mot Hundvåg
58272V2583663  -  Granstunnelen sørgående
21565V1060643  -  JORA
27436V2172082  -  HANEKLEIV SYD
80565B1689290  -  Nenset sykkeltellepunkt
22500V805651  -  HOVLAND
73380V121327  -  Rampe mot Prestheia
53155V1665289  -  RULLESTADTUNNELEN
97687V72808  -  Horg
27227V885156  -  Trældaltunnelen
72741V1811592  -  DALKRYSSET AVRAMPE
78805V2347241  -  REVET
21381V2554057  -  JAREN
14551V705197  -  NORDSTUMOEN S
71241V2460301  -  Kong Håkon 5.s gt Nordgående
18523V2282344  -  Ørbekk Nordgående